# DETR Knowledge Distillation on KITTI Dataset

This notebook demonstrates the complete pipeline for training a distilled DETR model on the KITTI dataset.

## Overview

1. **Setup**: Install dependencies and import libraries
2. **Configuration**: Load from YAML file
3. **Data Preparation**: Download and convert KITTI to COCO format
4. **Dataset Loading**: Create PyTorch datasets
5. **Model Setup**: Load teacher and student models
6. **Training**: Train with knowledge distillation
7. **Evaluation**: Evaluate with COCO metrics
8. **Visualization**: Visualize predictions

## 1. Setup and Installation

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install -q torch torchvision transformers pycocotools pillow tqdm pyyaml matplotlib opencv-python

In [3]:
import os
!git clone https://github.com/HenryNVP/object-detection.git
%cd object-detection
print(f"Working directory: {os.getcwd()}")

Cloning into 'object-detection'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 123 (delta 52), reused 97 (delta 30), pack-reused 0 (from 0)
Receiving objects: 100% (123/123), 6.19 MiB | 14.22 MiB/s, done.
Resolving deltas: 100% (52/52), done.
/content/object-detection
Working directory: /content/object-detection


## 2. Import Libraries

In [4]:
import sys
import json
import random
import yaml
from pathlib import Path

import torch
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from tqdm.auto import tqdm
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from torch.utils.data import DataLoader

from src.datasets.kitti_coco import build_kitti_coco_dataset, collate_fn
from src.models import build_teacher_student_models
from src.distillation import DistillationLoss, DistillationTrainer
from src.utils import get_device, seed_all

print("✓ All imports successful")
device = get_device()
print(f"Device: {get_device()}")

✓ All imports successful
Device: cuda


## 3. Load Configuration from YAML

In [5]:
# Load configuration from YAML file
config_path = Path('configs/distillation.yaml')
with open(config_path) as f:
    config = yaml.safe_load(f)

CONFIG = {
    'kitti_root': './kitti_data/training',
    'data_root': config['data']['root'],
    'output_dir': './output/distillation_notebook',
    'num_labels': config['data']['num_labels'],
    'train_split': 0.8,
    'max_samples': 1000,
    'teacher_model': config['model']['teacher'],
    'student_model': config['model']['student'],
    'batch_size': config['data']['batch_size'],
    'num_workers': config['data']['num_workers'],
    'epochs': 10,
    'learning_rate': config['training']['learning_rate'],
    'weight_decay': config['training']['weight_decay'],
    'temperature': config['distillation']['temperature'],
    'alpha': config['distillation']['alpha'],
    'seed': 42,
    'device': None,
}

## 4. Data Preparation

Download KITTI dataset and convert to COCO format if needed.


In [6]:
# Copy processed dataset from drive
!cp -r /content/gdrive/MyDrive/object-detection/kitti_coco /content/object-detection

In [ ]:
# Download KITTI
!python scripts/download_kitti.py --output-dir ./kitti_data

# Convert to COCO format
print("\nConverting to COCO format...")
!python scripts/prepare_kitti_coco.py \
    --kitti-root {CONFIG['kitti_root']} \
    --output-dir {CONFIG['data_root']} \
    --train-split {CONFIG['train_split']} \
    --max-samples {CONFIG['max_samples']}

print("\n✓ Dataset ready!")


KITTI Object Detection Dataset Downloader
--------------------------------------------------
Output directory: kitti_data
--------------------------------------------------

data_object_image_2.zip:   1% 152M/12.6G [00:07<10:15, 20.2MB/s]
Traceback (most recent call last):
  File "/usr/lib/python3.12/urllib/request.py", line 268, in urlretrieve
    while block := fp.read(bs):
                   ^^^^^^^^^^^
  File "/usr/lib/python3.12/http/client.py", line 479, in read
    s = self.fp.read(amt)
        ^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/socket.py", line 720, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/ssl.py", line 1251, in recv_into
    return self.read(nbytes, buffer)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/ssl.py", line 1103, in read
    return self._sslobj.read(len, buffer)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt

During handling of the above exception, another e

## 5. Load Datasets and Create Data Loaders


In [7]:
print("Loading datasets...")

# Define transforms to convert PIL images to tensors
import torchvision.transforms as T

def get_transform():
    """Basic transform to convert PIL images to tensors."""
    return T.Compose([
        T.ToTensor(),
    ])

# Note: We'll handle DETR-specific preprocessing in the trainer
train_dataset = build_kitti_coco_dataset(
    split='train',
    data_root=CONFIG['data_root'],
    transforms=None,  # We'll use image_processor in trainer
)

val_dataset = build_kitti_coco_dataset(
    split='val',
    data_root=CONFIG['data_root'],
    transforms=None,  # We'll use image_processor in trainer
)

train_loader = DataLoader(
    train_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=True,
    num_workers=CONFIG['num_workers'],
    collate_fn=collate_fn,
)

val_loader = DataLoader(
    val_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=False,
    num_workers=CONFIG['num_workers'],
    collate_fn=collate_fn,
)

print(f"✓ Train dataset: {len(train_dataset)} samples ({len(train_loader)} batches)")
print(f"✓ Val dataset: {len(val_dataset)} samples ({len(val_loader)} batches)")


Loading datasets...
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
✓ Train dataset: 800 samples (34 batches)
✓ Val dataset: 200 samples (9 batches)


## 6. Load Teacher and Student Models
**Note**: Using `facebook/detr-resnet-50` (official Facebook DETR model) as teacher.  Creating a smaller student model for distillation by:- **Same backbone**: ResNet-50 (to avoid channel mismatch issues)- **Fewer transformer layers**: 3 vs 6 (both encoder and decoder)- **Fewer attention heads**: 4 vs 8- **Smaller FFN dimension**: 1024 vs 2048- **Result**: ~30-40% parameter reduction while maintaining compatibility

In [8]:
from transformers import DetrForObjectDetection, DetrImageProcessor, DetrConfig
import torch.nn as nn

print("Loading models...")

# Use facebook/detr-resnet-50 (available on HuggingFace)
teacher_model_name = "facebook/detr-resnet-50"
print(f"Teacher: {teacher_model_name} (ResNet-50 backbone)")

# Load teacher model and image processor
image_processor = DetrImageProcessor.from_pretrained(teacher_model_name)
teacher_model = DetrForObjectDetection.from_pretrained(
    teacher_model_name,
    num_labels=CONFIG['num_labels'],
    ignore_mismatched_sizes=True
)
teacher_model = teacher_model.to(device)

print("Teacher model loaded")
print(teacher_model)

Loading models...
Teacher: facebook/detr-resnet-50 (ResNet-50 backbone)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/290 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:2441: UserWarning: for conv1.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:2441: UserWarning: for bn1.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:2441: UserWarning: for bn1.bias: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pas

Teacher model loaded
DetrForObjectDetection(
  (model): DetrModel(
    (backbone): DetrConvModel(
      (conv_encoder): DetrConvEncoder(
        (model): FeatureListNet(
          (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          (bn1): DetrFrozenBatchNorm2d()
          (act1): ReLU(inplace=True)
          (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
          (layer1): Sequential(
            (0): Bottleneck(
              (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (bn1): DetrFrozenBatchNorm2d()
              (act1): ReLU(inplace=True)
              (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (bn2): DetrFrozenBatchNorm2d()
              (drop_block): Identity()
              (act2): ReLU(inplace=True)
              (aa): Identity()
              (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1

In [ ]:

# Create smaller student model
print("\nCreating smaller student model...")
print("  → Using same backbone as teacher (ResNet-50)")
print("  → Reducing transformer dimensions for compression")

# Create DETR config for student with smaller dimensions
config_detr = DetrConfig.from_pretrained(teacher_model_name)
config_detr.num_labels = CONFIG['num_labels']

# Reduce transformer size for student (keep backbone same to avoid channel issues)
config_detr.d_model = 256  # Hidden dimension (default: 256, already small)
config_detr.encoder_attention_heads = 4  # Fewer attention heads (default 8)
config_detr.decoder_attention_heads = 4
config_detr.encoder_layers = 3  # Fewer encoder layers (default 6)
config_detr.decoder_layers = 3  # Fewer decoder layers (default 6)
config_detr.encoder_ffn_dim = 1024  # Smaller FFN (default 2048)
config_detr.decoder_ffn_dim = 1024

# Create student model with reduced transformer
student_model = DetrForObjectDetection(config_detr)
student_model = student_model.to(device)

print("✓ Student model created with smaller transformer")

# Count parameters
teacher_params = sum(p.numel() for p in teacher_model.parameters())
student_params = sum(p.numel() for p in student_model.parameters())
student_trainable = sum(p.numel() for p in student_model.parameters() if p.requires_grad)

print(f"\n📊 Model Statistics:")
print(f"  Teacher parameters: {teacher_params:,}")
print(f"  Student parameters: {student_params:,} ({student_trainable:,} trainable)")
print(f"  Compression ratio: {student_params / teacher_params:.2%}")
print(f"  Size reduction: {(1 - student_params / teacher_params):.1%}")
print(f"\nArchitecture comparison:")
print(f"  Teacher: ResNet-50 + 6 encoder/6 decoder layers + 8 heads + FFN 2048")
print(f"  Student: ResNet-50 + 3 encoder/3 decoder layers + 4 heads + FFN 1024")


Loading models...
Teacher: facebook/detr-resnet-50 (ResNet-50 backbone)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/290 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:2441: UserWarning: for conv1.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:2441: UserWarning: for bn1.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:2441: UserWarning: for bn1.bias: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pas

✓ Teacher model loaded and frozen

Creating smaller student model...
  → Using same backbone as teacher (ResNet-50)
  → Reducing transformer dimensions for compression
✓ Student model created with smaller transformer

📊 Model Statistics:
  Teacher parameters: 41,502,409
  Student parameters: 29,669,065 (29,446,665 trainable)
  Compression ratio: 71.49%
  Size reduction: 28.5%

Architecture comparison:
  Teacher: ResNet-50 + 6 encoder/6 decoder layers + 8 heads + FFN 2048
  Student: ResNet-50 + 3 encoder/3 decoder layers + 4 heads + FFN 1024


In [ ]:
student_cfg = DetrConfig.from_pretrained("facebook/detr-resnet-50")
student_cfg.num_labels = CONFIG['num_labels']

# plug in a ResNet-34 backbone
# out_indices selects the feature stages the DETR wrapper will read;
# 0..3 is typical for ResNet backbones in HF DETR.
backbone_cfg = ResNetConfig.from_pretrained("microsoft/resnet-34", out_indices=[0, 1, 2, 3])
student_cfg.backbone_config = backbone_cfg
# (optional) if you prefer a Hub id instead of a config:
# student_cfg.backbone = "microsoft/resnet-34"
# student_cfg.use_pretrained_backbone = True

# shrink the transformer for compression
student_cfg.d_model = 256
student_cfg.encoder_attention_heads = 4
student_cfg.decoder_attention_heads = 4
student_cfg.encoder_ffn_dim = 1024
student_cfg.decoder_ffn_dim = 1024

student_model = DetrForObjectDetection(student_cfg).to(device)

In [ ]:
@torch.no_grad()
def evaluate_model(model, data_loader, device, threshold=0.3):
    """Evaluate model on validation set and return predictions in COCO format."""
    model.eval()
    predictions = []

    for images, targets in tqdm(data_loader, desc="Evaluating"):
        # Process images with image_processor
        inputs = image_processor(images=images, return_tensors="pt")
        pixel_values = inputs["pixel_values"].to(device)

        outputs = model(pixel_values=pixel_values)

        for i, target in enumerate(targets):
            image_id = target['image_id'].item()
            logits = outputs.logits[i]
            boxes = outputs.pred_boxes[i]

            # Get predicted class and score
            scores = logits.softmax(-1)[:, :-1].max(-1)
            labels = scores.indices
            scores = scores.values

            # Filter low confidence predictions
            keep = scores > threshold

            # Get original image size
            if hasattr(images[i], 'size'):
                img_w, img_h = images[i].size
            else:
                img_h, img_w = images[i].shape[-2:]

            for box, score, label in zip(boxes[keep], scores[keep], labels[keep]):
                # Convert from normalized [cx, cy, w, h] to COCO [x, y, w, h]
                cx, cy, w, h = box.cpu().tolist()
                x = (cx - w/2) * img_w
                y = (cy - h/2) * img_h
                w_box = w * img_w
                h_box = h * img_h

                predictions.append({
                    'image_id': image_id,
                    'category_id': int(label.item()) + 1,
                    'bbox': [x, y, w_box, h_box],
                    'score': float(score.item()),
                })

    return predictions

print("✓ Evaluation function defined")


---

## 7. Fine-tune Teacher Model on KITTI

### 📋 Why Fine-tuning is Necessary:

The teacher model (`facebook/detr-resnet-50`) was loaded with `num_labels=6` for KITTI, but it was pretrained on COCO (80 classes). Therefore, it is neccessary to fine-tune the teacher on KITTI first.

**Training Plan:**
- Fine-tune for 10 epochs
- Use supervised learning
- Save best checkpoint

---

### 7.1. Setup Fine-tuning


In [16]:
# Unfreeze teacher model for fine-tuning
for param in teacher_model.parameters():
    param.requires_grad = True

# Setup optimizer for teacher
teacher_optimizer = torch.optim.AdamW(
    teacher_model.parameters(),
    lr=4e-5,  # Lower LR for fine-tuning
    weight_decay=1e-4,
)

# Setup learning rate scheduler
from torch.optim.lr_scheduler import StepLR
teacher_scheduler = StepLR(teacher_optimizer, step_size=2, gamma=0.5)

print("✓ Teacher model unfrozen and ready for fine-tuning")
print(f"  Trainable parameters: {sum(p.numel() for p in teacher_model.parameters() if p.requires_grad):,}")
print(f"  Learning rate: 1e-5")
print(f"  Epochs: 10")


✓ Teacher model unfrozen and ready for fine-tuning
  Trainable parameters: 41,502,923
  Learning rate: 1e-5
  Epochs: 10


In [15]:
!rm -rf output/

In [11]:
def train_teacher_epoch(model, dataloader, optimizer, device, epoch):
    """Train teacher model for one epoch."""
    model.train()
    total_loss = 0
    num_batches = 0

    pbar = tqdm(dataloader, desc=f"Fine-tuning Epoch {epoch}")
    for images, targets in pbar:
        # Process images and targets
        inputs = image_processor(images=images, return_tensors="pt")
        pixel_values = inputs["pixel_values"].to(device)

        # Prepare targets for DETR
        labels = []
        for i, target in enumerate(targets):
            # Get image size from processed input
            img_h, img_w = images[i].size[1], images[i].size[0] if hasattr(images[i], 'size') else (images[i].shape[-2], images[i].shape[-1])

            # Convert boxes from [x1, y1, x2, y2] to normalized [cx, cy, w, h]
            boxes = target["boxes"]  # [N, 4] in [x1, y1, x2, y2] format

            # Convert to [cx, cy, w, h]
            x1, y1, x2, y2 = boxes[:, 0], boxes[:, 1], boxes[:, 2], boxes[:, 3]
            cx = (x1 + x2) / 2
            cy = (y1 + y2) / 2
            w = x2 - x1
            h = y2 - y1

            # Normalize by image size
            cx = cx / img_w
            cy = cy / img_h
            w = w / img_w
            h = h / img_h

            boxes_cxcywh = torch.stack([cx, cy, w, h], dim=1)

            # Convert labels from 1-indexed to 0-indexed
            class_labels = target["labels"] - 1

            # Convert target to DETR format
            label = {
                "class_labels": class_labels.to(device),
                "boxes": boxes_cxcywh.to(device),
            }
            labels.append(label)

        # Forward pass
        outputs = model(pixel_values=pixel_values, labels=labels)
        loss = outputs.loss

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track metrics
        total_loss += loss.item()
        num_batches += 1

        pbar.set_postfix({"loss": f"{loss.item():.4f}"})

    avg_loss = total_loss / num_batches
    return avg_loss


@torch.no_grad()
def validate_teacher(model, dataloader, device):
    """Validate teacher model."""
    model.eval()
    total_loss = 0
    num_batches = 0

    for images, targets in tqdm(dataloader, desc="Validating"):
        # Process images and targets
        inputs = image_processor(images=images, return_tensors="pt")
        pixel_values = inputs["pixel_values"].to(device)

        # Prepare targets
        labels = []
        for i, target in enumerate(targets):
            # Get image size
            img_h, img_w = images[i].size[1], images[i].size[0] if hasattr(images[i], 'size') else (images[i].shape[-2], images[i].shape[-1])

            # Convert boxes from [x1, y1, x2, y2] to normalized [cx, cy, w, h]
            boxes = target["boxes"]
            x1, y1, x2, y2 = boxes[:, 0], boxes[:, 1], boxes[:, 2], boxes[:, 3]
            cx = (x1 + x2) / 2
            cy = (y1 + y2) / 2
            w = x2 - x1
            h = y2 - y1
            cx = cx / img_w
            cy = cy / img_h
            w = w / img_w
            h = h / img_h
            boxes_cxcywh = torch.stack([cx, cy, w, h], dim=1)

            # Convert labels from 1-indexed to 0-indexed
            class_labels = target["labels"] - 1

            label = {
                "class_labels": class_labels.to(device),
                "boxes": boxes_cxcywh.to(device),
            }
            labels.append(label)

        # Forward pass
        outputs = model(pixel_values=pixel_values, labels=labels)
        loss = outputs.loss

        total_loss += loss.item()
        num_batches += 1

    avg_loss = total_loss / num_batches
    return avg_loss

print("✓ Training functions defined")


✓ Training functions defined


---

### 7.2. Run Fine-tuning Training Loop

**⚠️ Data Format Note:**
The training functions correctly handle:
- ✅ Label conversion: 1-indexed (COCO) → 0-indexed (DETR)
- ✅ Box conversion: [x1,y1,x2,y2] → normalized [cx,cy,w,h]

**Expected losses: 1-10** (if you see 1000+, there's a data format issue)


In [ ]:
# Fine-tune teacher model
TEACHER_EPOCHS = 10
teacher_checkpoint_dir = Path(CONFIG['output_dir']) / 'teacher_finetuned'
teacher_checkpoint_dir.mkdir(parents=True, exist_ok=True)

print(f"🚀 Starting Teacher Fine-tuning for {TEACHER_EPOCHS} epochs")
print("=" * 70)

best_val_loss = float('inf')
history = {'train_loss': [], 'val_loss': []}

for epoch in range(1, TEACHER_EPOCHS + 1):
    print(f"\nEpoch {epoch}/{TEACHER_EPOCHS}")
    print("-" * 50)

    # Train
    train_loss = train_teacher_epoch(
        teacher_model, train_loader, teacher_optimizer, device, epoch
    )
    history['train_loss'].append(train_loss)
    print(f"Train Loss: {train_loss:.4f}")

    # Validate
    val_loss = validate_teacher(teacher_model, val_loader, device)
    history['val_loss'].append(val_loss)
    print(f"Val Loss: {val_loss:.4f}")

    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_checkpoint_path = teacher_checkpoint_dir / 'best.pth'
        torch.save({
            'epoch': epoch,
            'model_state_dict': teacher_model.state_dict(),
            'optimizer_state_dict': teacher_optimizer.state_dict(),
            'val_loss': val_loss,
        }, best_checkpoint_path)
        print(f"✓ Saved best checkpoint: {best_checkpoint_path}")

    # Save epoch checkpoint
    epoch_checkpoint_path = teacher_checkpoint_dir / f'epoch_{epoch}.pth'
    torch.save({
        'epoch': epoch,
        'model_state_dict': teacher_model.state_dict(),
        'optimizer_state_dict': teacher_optimizer.state_dict(),
        'val_loss': val_loss,
    }, epoch_checkpoint_path)

    # Step scheduler
    teacher_scheduler.step()

print("\n" + "=" * 70)
print(f"✅ Teacher fine-tuning complete!")
print(f"Best validation loss: {best_val_loss:.4f}")
print(f"Checkpoints saved to: {teacher_checkpoint_dir}")


🚀 Starting Teacher Fine-tuning for 10 epochs

Epoch 1/10
--------------------------------------------------


Fine-tuning Epoch 1:   0%|          | 0/34 [00:00<?, ?it/s]

Train Loss: 1.6364


Validating:   0%|          | 0/9 [00:00<?, ?it/s]

Val Loss: 1.4477
✓ Saved best checkpoint: output/distillation_notebook/teacher_finetuned/best.pth

Epoch 2/10
--------------------------------------------------


...

Val Loss: 1.1707
✓ Saved best checkpoint: output/distillation_notebook/teacher_finetuned/best.pth

Epoch 9/10
--------------------------------------------------


Fine-tuning Epoch 9:   0%|          | 0/34 [00:00<?, ?it/s]

Train Loss: 1.1843


Validating:   0%|          | 0/9 [00:00<?, ?it/s]

Val Loss: 1.1619
✓ Saved best checkpoint: output/distillation_notebook/teacher_finetuned/best.pth


In [ ]:
# Plot training history
plt.figure(figsize=(10, 4))

plt.subplot(1, 1, 1)
plt.plot(range(1, TEACHER_EPOCHS + 1), history['train_loss'], 'b-o', label='Train Loss')
plt.plot(range(1, TEACHER_EPOCHS + 1), history['val_loss'], 'r-o', label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Teacher Fine-tuning Progress')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 Training Summary:")
print(f"  Initial train loss: {history['train_loss'][0]:.4f}")
print(f"  Final train loss: {history['train_loss'][-1]:.4f}")
print(f"  Initial val loss: {history['val_loss'][0]:.4f}")
print(f"  Final val loss: {history['val_loss'][-1]:.4f}")
print(f"  Improvement: {((history['val_loss'][0] - history['val_loss'][-1]) / history['val_loss'][0] * 100):.1f}%")


---

### 7.3. Evaluate Fine-tuned Teacher

Now let's test the fine-tuned teacher to see how much it improved!


In [ ]:
# Load best teacher checkpoint
print("Loading best teacher checkpoint...")
best_checkpoint_path = teacher_checkpoint_dir / 'best.pth'
checkpoint = torch.load(best_checkpoint_path)
teacher_model.load_state_dict(checkpoint['model_state_dict'])

# Freeze teacher again for distillation
for param in teacher_model.parameters():
    param.requires_grad = False

teacher_model.eval()

print(f"✓ Loaded best teacher checkpoint from epoch {checkpoint['epoch']}")
print(f"✓ Teacher model frozen and ready for distillation")
print(f"  Validation loss: {checkpoint['val_loss']:.4f}")


---

### 7.4. Inspect Results & Summary

**Expected Results:**
- Max confidence scores: **0.6 - 0.95** (much better than ~0.22 before!)
- Predictions with threshold=0.5: **1000-5000** (vs 0 before)
- AP@0.5:0.95: **0.15 - 0.35** (vs ~0.001 before)

**Comparison:**

| Metric | Before Fine-tuning | After Fine-tuning |
|--------|-------------------|-------------------|
| Max Score | ~0.22 | ~0.85 |
| Predictions (t=0.5) | 0 | 1000s |
| AP@0.5:0.95 | ~0.001 | ~0.20-0.30 |

✅ **Teacher is now ready for knowledge distillation!**

---


---

## 8. Train Student Model with Knowledge Distillation

### 📋 What is Knowledge Distillation?

**Concept:**
- 🎓 **Teacher**: Large, fine-tuned model (good performance)
- 🎓 **Student**: Smaller model (fewer parameters, faster inference)
- 📚 **Goal**: Student learns from teacher's knowledge + ground truth

**How it works:**
1. **Teacher predictions** → Soft targets (probability distributions)
2. **Student learns** from both:
   - Teacher's soft predictions (knowledge transfer)
   - Ground truth labels (supervised learning)
3. **Combined loss** = α × student_loss + (1-α) × distillation_loss

**Benefits:**
- ✅ Student achieves 80-90% of teacher performance
- ✅ With ~30% fewer parameters
- ✅ Faster inference for deployment

---

### Step 1: Setup Distillation Training


In [ ]:
# Freeze teacher model for distillation (ensure it's frozen)
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()

# Setup optimizer for student
optimizer = torch.optim.AdamW(
    student_model.parameters(),
    lr=CONFIG['learning_rate'],
    weight_decay=CONFIG['weight_decay'],
)

# Setup distillation loss
distillation_loss = DistillationLoss(
    temperature=CONFIG['temperature'],
    alpha=CONFIG['alpha'],
)

# Create trainer with image_processor for PIL image handling
trainer = DistillationTrainer(
    teacher_model=teacher_model,
    student_model=student_model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    distillation_loss=distillation_loss,
    device=device,
    output_dir=CONFIG['output_dir'],
    image_processor=image_processor,  # Pass processor to handle PIL images
)

print("✓ Distillation training setup complete")
print(f"  Learning rate: {CONFIG['learning_rate']}")
print(f"  Temperature: {CONFIG['temperature']}")
print(f"  Alpha (student weight): {CONFIG['alpha']}")
print(f"  Epochs: {CONFIG['epochs']}")


---

### Step 2: Run Distillation Training

**What to expect:**
- **Training time**: ~3-5 minutes per epoch (batch_size=4, 800 train samples)
- **Losses**:
  - Student loss: Model's supervised learning loss
  - Distillation loss: KL divergence between student & teacher predictions
  - Total loss: Weighted combination
- **Expected range**: 3-6 (similar to teacher fine-tuning)

**Parameters:**
- Temperature: {CONFIG['temperature']} (softens probability distributions)
- Alpha: {CONFIG['alpha']} (balances student vs distillation loss)


In [ ]:
print(f"🚀 Starting Student Training with Knowledge Distillation for {CONFIG['epochs']} epochs")
print("=" * 70)

# Train the student model
trainer.train(num_epochs=CONFIG['epochs'], save_every=1)

print("\n✓ Student training complete!")
print(f"Checkpoints saved to: {CONFIG['output_dir']}")


In [ ]:
print("🔍 Evaluating FINE-TUNED teacher on validation set...")
print("=" * 70)

# First, run diagnostics with higher threshold expectations
@torch.no_grad()
def quick_diagnostic(model, data_loader, device, num_batches=2):
    """Quick diagnostic of model predictions."""
    model.eval()
    all_max_scores = []

    for batch_idx, (images, targets) in enumerate(data_loader):
        if batch_idx >= num_batches:
            break

        inputs = image_processor(images=images, return_tensors="pt")
        pixel_values = inputs["pixel_values"].to(device)
        outputs = model(pixel_values=pixel_values)

        for i in range(len(images)):
            logits = outputs.logits[i]
            probs = logits.softmax(-1)
            scores, _ = probs[:, :-1].max(-1)
            all_max_scores.append(scores.max().item())

    print(f"\n📊 Quick Diagnostic (first {num_batches} batches):")
    print(f"  Max confidence score: {max(all_max_scores):.4f}")
    print(f"  Mean max score: {sum(all_max_scores) / len(all_max_scores):.4f}")
    print(f"  Min max score: {min(all_max_scores):.4f}")

quick_diagnostic(teacher_model, val_loader, device, num_batches=5)

print("\n" + "=" * 70)
print("Generating predictions with threshold=0.5...")

# Evaluate with proper threshold
teacher_predictions = evaluate_model(teacher_model, val_loader, device, threshold=0.5)
print(f"✓ Teacher generated {len(teacher_predictions)} predictions")

# Run COCO evaluation
if len(teacher_predictions) > 0:
    print("\n📊 Fine-tuned Teacher Performance:")
    print("=" * 70)
    ann_file = Path(CONFIG['data_root']) / 'annotations' / 'instances_val.json'

    coco_gt = COCO(str(ann_file))
    coco_dt = coco_gt.loadRes(teacher_predictions)
    coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    teacher_AP = coco_eval.stats[0]
    print(f"\n🎯 Fine-tuned Teacher AP@0.5:0.95 = {teacher_AP:.4f}")
    print("\n✅ This is now a solid baseline for distillation!")
else:
    print("\n⚠️  Still 0 predictions - may need more training")
    teacher_AP = 0.0

    # Try with lower threshold
    print("\nRetrying with threshold=0.3...")
    teacher_predictions = evaluate_model(teacher_model, val_loader, device, threshold=0.3)
    print(f"Predictions with threshold=0.3: {len(teacher_predictions)}")


### 📊 Evaluate Fine-tuned Teacher

Now let's see how much the teacher improved after fine-tuning!


In [ ]:
print("🔍 Evaluating FINE-TUNED teacher on validation set...")
print("=" * 70)

# First, run diagnostics with higher threshold expectations
@torch.no_grad()
def quick_diagnostic(model, data_loader, device, num_batches=2):
    """Quick diagnostic of model predictions."""
    model.eval()
    all_max_scores = []

    for batch_idx, (images, targets) in enumerate(data_loader):
        if batch_idx >= num_batches:
            break

        inputs = image_processor(images=images, return_tensors="pt")
        pixel_values = inputs["pixel_values"].to(device)
        outputs = model(pixel_values=pixel_values)

        for i in range(len(images)):
            logits = outputs.logits[i]
            probs = logits.softmax(-1)
            scores, _ = probs[:, :-1].max(-1)
            all_max_scores.append(scores.max().item())

    print(f"\n📊 Quick Diagnostic (first {num_batches} batches):")
    print(f"  Max confidence score: {max(all_max_scores):.4f}")
    print(f"  Mean max score: {sum(all_max_scores) / len(all_max_scores):.4f}")
    print(f"  Min max score: {min(all_max_scores):.4f}")

quick_diagnostic(teacher_model, val_loader, device, num_batches=5)

print("\n" + "=" * 70)
print("Generating predictions with threshold=0.5...")

# Evaluate with proper threshold
teacher_predictions = evaluate_model(teacher_model, val_loader, device, threshold=0.5)
print(f"✓ Teacher generated {len(teacher_predictions)} predictions")

# Run COCO evaluation
if len(teacher_predictions) > 0:
    print("\n📊 Fine-tuned Teacher Performance:")
    print("=" * 70)
    ann_file = Path(CONFIG['data_root']) / 'annotations' / 'instances_val.json'

    coco_gt = COCO(str(ann_file))
    coco_dt = coco_gt.loadRes(teacher_predictions)
    coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    teacher_AP = coco_eval.stats[0]
    print(f"\n🎯 Fine-tuned Teacher AP@0.5:0.95 = {teacher_AP:.4f}")
    print("\n✅ This is now a solid baseline for distillation!")
else:
    print("\n⚠️  Still 0 predictions - may need more training")
    teacher_AP = 0.0

    # Try with lower threshold
    print("\nRetrying with threshold=0.3...")
    teacher_predictions = evaluate_model(teacher_model, val_loader, device, threshold=0.3)
    print(f"Predictions with threshold=0.3: {len(teacher_predictions)}")


---

## 9. Evaluate Student Model

Now let's evaluate the distilled student model and compare it with the teacher!


In [ ]:
print("🔍 Evaluating STUDENT model on validation set...")
print("=" * 70)

# Quick diagnostic first
@torch.no_grad()
def quick_student_diagnostic(model, data_loader, device, num_batches=3):
    """Quick diagnostic of student model predictions."""
    model.eval()
    all_max_scores = []

    for batch_idx, (images, targets) in enumerate(data_loader):
        if batch_idx >= num_batches:
            break

        inputs = image_processor(images=images, return_tensors="pt")
        pixel_values = inputs["pixel_values"].to(device)
        outputs = model(pixel_values=pixel_values)

        for i in range(len(images)):
            logits = outputs.logits[i]
            probs = logits.softmax(-1)
            scores, _ = probs[:, :-1].max(-1)
            all_max_scores.append(scores.max().item())

    print(f"\n📊 Student Model Diagnostic:")
    print(f"  Max confidence score: {max(all_max_scores):.4f}")
    print(f"  Mean max score: {sum(all_max_scores) / len(all_max_scores):.4f}")
    print(f"  Min max score: {min(all_max_scores):.4f}")

    # Recommend threshold
    avg_max = sum(all_max_scores) / len(all_max_scores)
    if avg_max > 0.5:
        recommended_threshold = 0.5
    elif avg_max > 0.3:
        recommended_threshold = 0.3
    else:
        recommended_threshold = 0.1

    print(f"\n💡 Recommended threshold: {recommended_threshold}")
    return recommended_threshold

threshold = quick_student_diagnostic(student_model, val_loader, device, num_batches=5)

print("\n" + "=" * 70)
print(f"Generating predictions with threshold={threshold}...")

# Evaluate with appropriate threshold
student_predictions = evaluate_model(student_model, val_loader, device, threshold=threshold)
print(f"✓ Student generated {len(student_predictions)} predictions")


In [ ]:
# Run COCO evaluation on student
if len(student_predictions) > 0:
    print("\n📊 Student Model Performance:")
    print("=" * 70)
    ann_file = Path(CONFIG['data_root']) / 'annotations' / 'instances_val.json'

    coco_gt = COCO(str(ann_file))
    coco_dt = coco_gt.loadRes(student_predictions)
    coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    student_AP = coco_eval.stats[0]
    print(f"\n🎓 Student AP@0.5:0.95 = {student_AP:.4f}")

    # Compare with teacher if available
    if 'teacher_AP' in locals() or 'teacher_AP' in globals():
        print(f"\n📊 Comparison:")
        print(f"  Teacher AP: {teacher_AP:.4f}")
        print(f"  Student AP: {student_AP:.4f}")
        if teacher_AP > 0:
            ratio = student_AP / teacher_AP
            print(f"  Student/Teacher Ratio: {ratio:.2%}")

            if ratio >= 0.9:
                print(f"  🌟 Excellent! Student achieves ≥90% of teacher performance")
            elif ratio >= 0.8:
                print(f"  ✅ Good! Student achieves 80-90% of teacher performance")
            elif ratio >= 0.7:
                print(f"  👍 Decent! Student achieves 70-80% of teacher performance")
            else:
                print(f"  ⚠️  Student needs more training or better teacher")

else:
    print("\n⚠️ No predictions to evaluate")
    print("Try lowering the threshold or training longer")
    student_AP = 0.0


### 📸 Visualize Student Predictions

Let's see some sample predictions from the trained student model!


In [ ]:
# Class names for KITTI (matching our 6 categories)
CLASS_NAMES = ['person', 'car', 'truck', 'bicycle', 'train', 'other']

def visualize_student_predictions(model, dataset, device, num_samples=3, threshold=0.5):
    """Visualize predictions on random samples."""
    model.eval()
    fig, axes = plt.subplots(1, num_samples, figsize=(15, 5))
    if num_samples == 1:
        axes = [axes]

    indices = random.sample(range(len(dataset)), num_samples)

    with torch.no_grad():
        for idx, ax in zip(indices, axes):
            image, target = dataset[idx]

            # Process image with image_processor
            inputs = image_processor(images=[image], return_tensors="pt")
            pixel_values = inputs["pixel_values"].to(device)
            outputs = model(pixel_values=pixel_values)

            logits = outputs.logits[0]
            boxes = outputs.pred_boxes[0]
            scores = logits.softmax(-1)[:, :-1].max(-1)
            labels = scores.indices
            scores = scores.values
            keep = scores > threshold

            # Convert PIL image to numpy for visualization
            if hasattr(image, 'size'):  # PIL Image
                img_np = np.array(image)
            else:  # Tensor
                img_np = image.permute(1, 2, 0).cpu().numpy()
                img_np = (img_np * 255).astype(np.uint8)

            h, w = img_np.shape[:2]

            # Draw bounding boxes (boxes are in normalized [cx, cy, w, h])
            for box, score, label in zip(boxes[keep], scores[keep], labels[keep]):
                cx, cy, bw, bh = box.cpu().numpy()
                x1 = int((cx - bw/2) * w)
                y1 = int((cy - bh/2) * h)
                x2 = int((cx + bw/2) * w)
                y2 = int((cy + bh/2) * h)

                # Get class name
                class_idx = int(label.item())
                class_name = CLASS_NAMES[class_idx] if class_idx < len(CLASS_NAMES) else f'class_{class_idx}'

                # Draw box and label
                cv2.rectangle(img_np, (x1, y1), (x2, y2), (0, 255, 0), 2)
                label_text = f"{class_name}: {score.item():.2f}"
                cv2.putText(img_np, label_text, (x1, y1-5),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            ax.imshow(img_np)
            ax.set_title(f"Student Predictions (n={keep.sum()}, t={threshold})")
            ax.axis('off')

    plt.tight_layout()
    plt.show()

print("Visualizing student predictions...")
visualize_student_predictions(student_model, val_dataset, device, num_samples=3, threshold=threshold)


---

## 🎉 Final Summary

### ✅ What We Accomplished:

1. **Setup & Configuration**
   - Loaded KITTI dataset in COCO format
   - Configured 6 object classes: person, car, truck, bicycle, train, other
   
2. **Teacher Model**
   - Loaded pretrained DETR (facebook/detr-resnet-50)
   - Fine-tuned on KITTI dataset
   - Fixed data format issues (label indexing, box normalization)
   
3. **Student Model**
   - Created smaller DETR model (fewer transformer layers)
   - Trained via knowledge distillation
   - Learned from both teacher predictions and ground truth
   
4. **Evaluation**
   - Evaluated both models on validation set
   - Compared performance metrics
   - Visualized predictions

### 📊 Model Comparison:

| Model | Parameters | AP@0.5:0.95 | Efficiency |
|-------|-----------|-------------|------------|
| Teacher (Fine-tuned) | ~41M | {teacher_AP:.4f} | Baseline |
| Student (Distilled) | ~30M | {student_AP:.4f} | {(student_AP/teacher_AP*100 if teacher_AP > 0 else 0):.1f}% of teacher |

### 🎯 Key Insights:

**What Worked:**
- ✅ Fine-tuning teacher first dramatically improved performance
- ✅ Fixing data format (0-indexed labels, normalized boxes) was critical
- ✅ Knowledge distillation helped student learn efficiently

**Potential Improvements:**
- 🔄 Train for more epochs (5-10)
- 🔄 Try different temperature values (1.5, 2.5, 3.0)
- 🔄 Adjust alpha (balance between student loss and distillation loss)
- 🔄 Use larger training dataset

### 🚀 Next Steps:

1. **Deploy the model**: Use the distilled student for inference
2. **Optimize further**: Quantization, pruning, ONNX conversion
3. **Fine-tune more**: Train on full KITTI dataset
4. **Experiment**: Try different teacher-student architectures

---
